## 1: Implementation of Exact CNTK

In [1]:
import numpy as np
import torch

import TDL
from TDL.data import data_loader
from TDL.models import CNTK
from TDL.metrics import metrics
from TDL.estimation import estimation

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

torch.manual_seed(42)
np.random.seed(42)

Using device: cuda


In [2]:
# Define train and test sizes
n_train = 200
n_test = 100

X_train, Y_train, X_test, Y_test, y_test_labels = data_loader.load_cifar10(n_train=n_train, n_test=n_test)

Data loaded: Train torch.Size([200, 3, 32, 32]), Test torch.Size([100, 3, 32, 32])
Label range: [-0.1000, 0.9000]


In [3]:
# Select model depth
depth = 6

In [4]:
# Run experiments
results = {'Depth': depth}

# CNTK-V (Vanilla)
acc_v, time_v = estimation.run_experiment(X_train, Y_train, X_test, y_test_labels,
                               depth=depth, use_gap=False)
results['CNTK-V'] = acc_v

# CNTK-GAP
acc_gap, time_gap = estimation.run_experiment(X_train, Y_train, X_test, y_test_labels,
                                    depth=depth, use_gap=True)
results['CNTK-GAP'] = acc_gap


Running: Depth=6, GAP=False
Computing training kernel...


Computing kernel: 100%|██████████| 20100/20100 [05:05<00:00, 65.90it/s]


Computing test kernel...


Computing kernel: 100%|██████████| 20000/20000 [09:25<00:00, 35.39it/s]


Kernel computation time: 870.34s
Accuracy: 25.00%

Running: Depth=6, GAP=True
Computing training kernel...


Computing kernel: 100%|██████████| 20100/20100 [10:40<00:00, 31.36it/s]


Computing test kernel...


Computing kernel: 100%|██████████| 20000/20000 [14:40<00:00, 22.71it/s]

Kernel computation time: 1521.56s
Accuracy: 37.00%


In [10]:
# Comparison with paper results
paper_results = [
    {
    'Depth': 11,
    'CNN-V': 69.48,
    'CNTK-V': 65.90,
    'CNTK-V-2K': 43.42,
    'CNN-GAP': 82.92,
    'CNTK-GAP': 77.43,
    'CNTK-GAP-2K': 51.92
    },
    {
    'Depth': 6,
    'CNN-V': 64.11,
    'CNTK-V': 66.03,
    'CNTK-V-2K': 43.42,
    'CNN-GAP': 83.75,
    'CNTK-GAP': 76.73,
    'CNTK-GAP-2K': 51.73
    }
]

current_depth = results['Depth']
matching_paper_result = next((item for item in paper_results if item['Depth'] == current_depth), {})

# Comparison table
print("RESULTS COMPARISON")
print("="*55)
print(f"{'Depth':<10} {'Method':15} {'Paper':<15} {'Reproduced':<15}")
print("-"*55)

for method in results:
    if method == 'Depth':
        continue
    
    paper_val = matching_paper_result.get(method, 'N/A')
    
    val = results[method]
    print(f"{current_depth:<10} {method:<15} {paper_val:<15} {val:.2f}%")
print("="*55)

RESULTS COMPARISON
Depth      Method          Paper           Reproduced     
-------------------------------------------------------
6          CNTK-V          66.03           25.00%
6          CNTK-GAP        76.73           37.00%
